Possible ideas for the plan:
Not entirely sure where we will use which technology(*vector,image-to text, etc.*), and also still need to finish the work division

0-load all necessary data/methods/pip installs

1-our application will first allow the user to state what scene they saw - using this input to feed to classification algorithm, trained on furniture dataset

2-internally it will categorize what possible room it could have been, we need tokenization and a classifier for that

3-It needs to relay which room it decides on - this means it needs to print output to the user

5-then it will teleport to that room, for which we will use ai2thor's Teleport() function

6-once teleported, it needs to communicate to the user if the current scene is similar to the scene that they were looking for

7-calculate a similarity score - using text to vector(*did we do this for room-comparison or scene-comparison?*)

for scene comparison

*   7.1-if score high: print image(*wouldnt it already do that after teleporting?*). State which objects are closeby, (*in case the object they needed to find is behind something and thus could not normally be seen*). We can use the listing function of ai2thor/image recognition
*   7.1.1-then let user say if the object is found

    *   7.1.2-if yes: say "great job, we did it" or something, end interaction
    *   7.1.3-if no: go to point 7.2

*   7.2-if score average: tell user to take controls to let the agent move around in the room manually

*   7.3-if score low: Say "did not find the place". Randomly teleport and start again at point 5

(the easiest way is just to teleport() randomly and start at point 5)

-friday presentation: both

-final presentation: both

-report: i think i can do most of that if its okay with you, i write quite fast.

as you like.

Notes: For similarity scores we might later need to adjust the level(from >90, to >95 or so if we find it neccessary).If maybe rotate then recalculate score. okay, let agent move/rotate not user. teleport does not need to be randomly. succes not most important. but recording the actions/communication and analyze communication between user/computer. both have descriptions and we also would know what is the quality of those. Also how well did similarity score perform. need to take into account user can say yes/no/alternative decription when asked questions by the agent, so would need to be able to handle such scenarios if we would have them.

In [2]:
import random
import warnings,logging
warnings.simplefilter('ignore')
logging.disable(logging.WARNING)

In [3]:
import image_text as caption
import room_classification as room
import text_to_vector as vec
from sklearn.metrics.pairwise import cosine_similarity


In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
# !pip install --upgrade ai2thor --quiet
# !pip install ai2thor-colab prior --upgrade &> /dev/null

In [6]:
import os
os.system('apt-get install xvfb')

import prior
dataset = prior.load_dataset("procthor-10k")
house = dataset["train"][11]

# import ai2thor_colab
# ai2thor_colab.start_xserver()

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


[AI2-THOR WARNING] There has been an update to ProcTHOR-10K that must be used with AI2-THOR version 5.0+. To use the new version of ProcTHOR-10K, please update AI2-THOR to version 5.0+ by running:
    pip install --upgrade ai2thor
Alternatively, to downgrade to the old version of ProcTHOR-10K, run:
   prior.load_dataset("procthor-10k", revision="ab3cacd0fc17754d4c080a3fd50b18395fae8647")


Loading test: 100%|██████████████████████| 1000/1000 [00:00<00:00, 12359.74it/s]


In [7]:
from ai2thor.controller import Controller
from ai2thor.platform import CloudRendering
from PIL import Image

controller = Controller(platform=CloudRendering)
img = Image.fromarray(controller.last_event.frame)
# controller = Controller(scene='FloorPlan30',platform=CloudRendering)
# controller = Controller(scene=house, visibilityDistance=1.5 , width=150, height=100)


In [8]:
def Teleporting():
    event = controller.step(action="GetReachablePositions")
    reachable_positions = event.metadata["actionReturn"]
    event.metadata["actionReturn"]
    position = random.choice(reachable_positions)
    rotation = random.choice(range(360))
    event = controller.step(action="Teleport", position=position, rotation=rotation)
    img = Image.fromarray(controller.last_event.frame)
    return img

def Low_score_actions(): #i changed this function so it doesnt take input anymore and lets the agent decide
    print('Close to the object, but i will rotate so we might find it.')
    controller.step("RotateRight")
    img = Image.fromarray(controller.last_event.frame)
    return img


In [9]:
# !pip install nltk

In [10]:
import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
import pandas as pd
from nltk.corpus import stopwords

In [11]:
def remove_stopwords_and_make_dataframe(caption):
    # Tokenize the generated text into words
    words = nltk.word_tokenize(caption)
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word.lower() not in stop_words]
    # Join the filtered words into a string
    filtered_text = ' '.join(filtered_words)
    return filtered_text

# run the robot

In [12]:
# Constants
COSINE_SIMILARITY_THRESHOLD = 0.93 #need to run the program a bunch of times to find suitable threshold values
CLOSE_OBJECTS_THRESHOLD_LOW = 0.83
CLOSE_OBJECTS_THRESHOLD_HIGH = 0.93
MAX_ITERATIONS = 15


# Directory to save images
image_dir = "found_images"

# Create the directory if it doesn't exist
os.makedirs(image_dir, exist_ok=True)

def main(human_text, img, org_img_path):
    NR_ROTATIONS = 0
    
    human_room_prediction = predict_room(human_text)
    human_text_embeddings = vec.encode_text_with_bert(human_text)
    find = False

    for count in range(MAX_ITERATIONS):
        generated_caption = caption.image_to_caption(img)['generated_text']
        robot_room_prediction = predict_room(generated_caption)
        robot_text_embeddings = vec.encode_text_with_bert(generated_caption)

        # Check if the robot is in the same room as the human
        if robot_room_prediction == human_room_prediction:
            # Check for similarity with input text
            similarity_score = cosine_similarity(human_text_embeddings, robot_text_embeddings)[0][0]

            if similarity_score >= COSINE_SIMILARITY_THRESHOLD:
                print("Cosine Similarity Score:", similarity_score)
                print("I'm in the place")
                find = True

                # Define the image filename
                image_filename = f"{org_img_path[7:-4:]}_found.jpg"
                
                # Save the image to the specified directory
                img.save(os.path.join(image_dir, image_filename))


                
                break
            elif CLOSE_OBJECTS_THRESHOLD_LOW <= similarity_score < CLOSE_OBJECTS_THRESHOLD_HIGH and NR_ROTATIONS < 4: #as long as we have turned max 3 times
                NR_ROTATIONS = NR_ROTATIONS + 1
                img = Low_score_actions() #we turn and add one to the NR_ROTATIONS
                continue 
            else:
                print("I'm not in the place, I'll teleport somewhere else")
                img = Teleporting()
        else:
            img = Teleporting()

    if find:
        # display(img)
        # Append the results to the DataFrame
        result_df.loc[len(result_df)] = [org_img_path, human_text, human_room_prediction,image_filename,generated_caption, robot_room_prediction, similarity_score]
        print("Results saved in DataFrame.")
    else:
        print('Did not find the place')


def predict_room(text):
    filtered_text = remove_stopwords_and_make_dataframe(text)
    model = room.load_best_model()
    prediction = model.predict([filtered_text])[0]
    return prediction[0]



# DataFrame initialization to save result
result_columns = ["Original Image", "Input Text",'user predicted room', "predicted place", "Generated Caption", 'robot predicted room' ,"Cosine Similarity Score"]
result_df = pd.DataFrame(columns=result_columns)

# Loading the evaluation data
file_path = 'data_evaluation.tsv'
df_eval = pd.read_csv(file_path, sep=';')


num_of_caption = 50

# Loop through the adjusted captions
for index, row in df_eval.head(num_of_caption).iterrows():
    
    # controller = Controller(platform=CloudRendering)
    img = Image.fromarray(controller.last_event.frame)

    input_text = row['adjusted Caption']
    org_img_path = row['Image Path']
    print(org_img_path,"::",'next')

    # run the simulation
    main(input_text, img,org_img_path)

# Save results
result_df.to_csv("place_found.csv")

images/1.jpg :: next


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 13.22it/s]


Close to the object, but i will rotate so we might find it.


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 12.03it/s]


I'm not in the place, I'll teleport somewhere else


...

I'm not in the place, I'll teleport somewhere else


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 14.19it/s]


I'm not in the place, I'll teleport somewhere else


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 13.10it/s]


Did not find the place


In [16]:
file_path = 'place_found.csv'
result = pd.read_csv(file_path)

result

,Unnamed: 0,Original Image,Input Text,user predicted room,predicted place,Generated Caption,robot predicted room,Cosine Similarity Score
0,0,images/8.jpg,"a kitchen equipped with a refrigerator, sink, ...",3,8_found.jpg,"a kitchen with a stove, sink, and refrigerator",3,0.967165
1,1,images/10.jpg,"a kitchen with a counter, sink, and a window",3,10_found.jpg,"a kitchen with a stove, sink, and a refrigerator",3,0.945124
2,2,images/20.jpg,"a kitchen with a microwave, sink, stove, and r...",3,20_found.jpg,"a kitchen with a stove, microwave, refrigerato...",3,0.962595
3,3,images/27.jpg,a room with a black and white floor and a whit...,1,27_found.jpg,a room with a white wall and a black floor,1,0.973317
4,4,images/28.jpg,a room with a black wall and a white floor,1,28_found.jpg,a room with a white wall and a black floor,1,0.999162
5,5,images/29.jpg,"a kitchen with a sink, refrigerator, and stove",3,29_found.jpg,"a kitchen with a sink, stove, and a window",3,0.948319
6,6,images/31.jpg,"a kitchen with a refrigerator, sink, and stove",3,31_found.jpg,"a kitchen with a stove, refrigerator, and a sink",3,0.983192
7,7,images/32.jpg,a room with a door and a wooden floor,1,32_found.jpg,a hallway with a wooden floor and a wooden floor,1,0.939449
8,8,images/33.jpg,a room with a window and a wall,1,33_found.jpg,a room with a door and a wall with a wooden fl...,1,0.944226
9,9,images/38.jpg,a room with a black and white floor and a whit...,1,38_found.jpg,a room with a white wall and a black and white...,1,0.995671
